In [1]:
# import the necessary packages
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_proc
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobileNet_proc
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2

In [2]:
# Variable to hold the mask, age and location information
maskInfo = []
ageInfo  = []
locInfo  = []

def detect_face_and_predict(frame, faceNet, maskNet, ageNet, full):
    #Allow use of global variables 
    global locInfo
    global maskInfo
    global ageInfo
    
    # Get the dim of the frame 
    (h, w) = frame.shape[:2]
    # Create the blob
    blob = cv2.dnn.blobFromImage(frame, 
                                 1.05,  # Scale factor 
                                 (224, 224), # Size 
                                 (104.0, 177.0, 123.0) # Scaler (mean-R, mean-G, mean-B) 
                                )
    
    # Get the face net and detect the face
    faceNet.setInput(blob)
    
    detectedData = faceNet.forward()
    
    # local variables to hold the temp faces and age data 
    faces    = []
    age      = []
    
    # Clear the loc info every time and mask/age info only when full is True 
    # so that mask/age is calculated only certain time as control by caller to speed up
    locInfo  = []
    if(full):
        maskInfo = []
        ageInfo  = []

    # loop over the different detected data
    for i in range(0, detectedData.shape[2]):
 
        # Get the Probability value from the data 
        probabilityValue  = detectedData[0, 0, i, 2]

        # Choose only high probability results
        if probabilityValue > 0.5:
            # Get the coordinates of the object 
            
            box = detectedData[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Make sure the values are withing frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # Extract the face and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            
            ageFace = np.copy(face)
            ageFace = vgg_proc(ageFace)
            ageFace = ageFace/255.0
            face = mobileNet_proc(face)
            
            # Add the faces into respective lists
            faces.append(face)
            age.append(ageFace)
            locInfo.append((startX, startY, endX, endY))

    # Check if face is there and full is True
    if len(faces) > 0 and full:
        faces = np.array(faces, dtype="float32")
        maskInfo = maskNet.predict(faces, batch_size=32)
        
        # Predict the age only if no mask is detected
        for i in range (len(maskInfo)):
            if maskInfo[i][1] > maskInfo[i][0]:
                img = age[i]
                img = np.expand_dims(img, axis=0)
                img_class = ageNet.predict(img)
                ageInfo.append(img_class[0][0])
            else:
                ageInfo.append(0)


In [ ]:
# Load face detector model 
prototxtFile = r"/home/hemlata/AML_maskdetection/deploy.prototxt"
weightsFile = r"/home/hemlata/AML_maskdetection/res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtFile, weightsFile)

# load the mask and age detector model
maskNet = load_model("/home/hemlata/AML_maskdetection/mobilenet.h5")

ageNet= load_model("/home/hemlata/AML_maskdetection/vgg16_agenet.h5")

# initialize the video stream
print("[INFO] starting video stream...")

# From Camera
#vs = cv2.VideoCapture(0 , cv2.CAP_V4L)

# 4 people 
#vs = cv2.VideoCapture('/home/hemlata/AML_maskdetection/mask.mp4')

# 2 people
#vs = cv2.VideoCapture('/home/hemlata/AML_maskdetection/Mask2.mp4')

# Walking
#vs = cv2.VideoCapture('/home/hemlata/AML_maskdetection/MaskWalk.mp4')
#vs = cv2.VideoCapture('/home/hemlata/AML_maskdetection/walkingpeople.mp4')
vs = cv2.VideoCapture('/home/hemlata/AML_maskdetection/manypeople.mp4')


count = 0
# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream 
    ret, frame = vs.read()

    # Check the frame and make predictions 
    detect_face_and_predict(frame, faceNet, maskNet, ageNet, count==0)
    count = 0 if count == 5 else count + 1
    
    # Loop each face and add label 
    for (box, maskinfo, age) in zip(locInfo, maskInfo, ageInfo):
        
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = maskinfo

        # Create label based on mask(Green label)/withoutMask(Red label) 
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        # Add the confidence and age into label based on mask/withoutMask 
        if mask < withoutMask:
            label = "{}: {:}% Age:{}".format(label, int(max(mask, withoutMask) * 100), int(age))
        else:
            label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # display the label and bounding box rectangle
        cv2.putText(frame, label, (startX, startY - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame
    frame = imutils.resize(frame, width=800)
    #frame = cv2.resize(frame, (1050, 920))
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

[INFO] starting video stream...
